In [238]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy import stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm as anova
import itertools
from sklearn import linear_model
from numpy import ones,vstack
from numpy.linalg import lstsq
from mpl_toolkits.axes_grid1 import make_axes_locatable
import osmnx as ox
import contextily as cx
import folium
from geopy.geocoders import Nominatim
import geopandas as gpd
import fiona
from arcgis.gis import GIS
from folium.plugins import MarkerCluster
from geopy.distance import geodesic

In [ ]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
schools = gpd.read_file('../GeoJsonFiles/schools_gmaps.kml', driver='KML')

In [ ]:
schools

https://geopandas.readthedocs.io/en/latest/gallery/plotting_basemap_background.html


In [ ]:
landuse_polygon = gpd.read_file('../GeoJsonFiles/landuse_polygon.geojson')
ames_shape = gpd.read_file('../GeoJsonFiles/shape/natural.shp')


In [226]:
### ALL THE BUILDINGS!!!
gis_buildings = gpd.read_file('../GeoJsonFiles/iowa-latest-free/gis_osm_buildings_a_free_1.shp')
gis_buildings

,osm_id,code,fclass,name,type,geometry
0,15834010,1500,building,None,detached,"POLYGON ((-95.16811 42.66000, -95.16811 42.660..."
1,15834016,1500,building,None,detached,"POLYGON ((-95.16569 42.66122, -95.16535 42.661..."
2,15834053,1500,building,None,None,"POLYGON ((-95.16729 42.66056, -95.16729 42.660..."
3,15878265,1500,building,None,None,"POLYGON ((-95.30954 43.45405, -95.30953 43.454..."
4,15895745,1500,building,None,None,"POLYGON ((-92.67338 43.06761, -92.67333 43.067..."
...,...,...,...,...,...,...
232496,975103891,1500,building,None,house,"POLYGON ((-95.65115 40.60407, -95.65112 40.604..."
232497,975103892,1500,building,None,house,"POLYGON ((-95.65174 40.60479, -95.65172 40.604..."
232498,975103893,1500,building,None,house,"POLYGON ((-95.65117 40.60461, -95.65112 40.604..."
232499,975103894,1500,building,None,garage,"POLYGON ((-95.65111 40.60443, -95.65107 40.604..."


In [155]:
gis_osm_poi = gpd.read_file('../GeoJsonFiles/iowa-latest-free/gis_osm_pois_free_1.shp')
poi_crs = gis_osm_poi.to_crs(epsg=3857)


In [231]:
gis_osm_poi
# poi_crs['fclass'].unique()
poi_crs

,osm_id,code,fclass,name,geometry
0,151468897,2742,viewpoint,Mount Lucia,POINT (-10738386.211 5242314.576)
1,151517465,2303,cafe,Kathy's Korner,POINT (-10581075.740 5206936.110)
2,151910372,2401,hotel,Arrowwood Resort Okoboji,POINT (-10589667.535 5371098.602)
3,159672331,2952,water_tower,None,POINT (-10338357.528 5307286.925)
4,160674079,2952,water_tower,None,POINT (-10540512.666 5328783.064)
...,...,...,...,...,...
16683,9018169875,2204,park,Lock & Dam 12 Recreation Area,POINT (-10065811.269 5199943.019)
16684,9021567946,2302,fast_food,None,POINT (-10189443.241 5109892.308)
16685,9021567947,2301,restaurant,None,POINT (-10189442.707 5109939.749)
16686,9021567948,2301,restaurant,None,POINT (-10189471.906 5109892.189)


In [252]:
### CHURCHES
# gis_osm_pofw = gpd.read_file('../GeoJsonFiles/iowa-latest-free/gis_osm_pofw_free_1.shp')
# pofw_crs = gis_osm_pofw.to_crs(epsg=3857)
gis_osm_pofw[gis_osm_pofw['fclass']=='jewish']

# data (list of list with values) – List of list of shape [[lat, lon], [lat, lon], etc.] 
# When you use a custom callback you could add more values after the lat and lon. 
# E.g. [[lat, lon, ‘red’], [lat, lon, ‘blue’]]

,osm_id,code,fclass,name,geometry,x,y,locations,lon,lat
50,354296458,3200,jewish,Bnai Moses Synagogue,POINT (-91.04126 41.42920),-91.041261,41.429195,"(41.4291952, -91.0412606)",-91.041261,41.429195
1094,354344435,3200,jewish,Adas Israel Synagogue,POINT (-93.20465 43.15802),-93.204648,43.158017,"(43.1580171, -93.2046479)",-93.204648,43.158017
1584,354355228,3200,jewish,B'Nai Israel Synagogue,POINT (-95.85250 41.26291),-95.852496,41.262912,"(41.2629125, -95.8524958)",-95.852496,41.262912
2518,354373395,3200,jewish,Temple Israel,POINT (-91.11044 40.80770),-91.110440,40.807703,"(40.8077032, -91.11044)",-91.110440,40.807703
2822,354380189,3200,jewish,Ames Jewish Congregation,POINT (-93.62553 42.05833),-93.625528,42.058327,"(42.0583267, -93.6255278)",-93.625528,42.058327
3410,354389878,3200,jewish,Congregation Beth Shalom,POINT (-96.39947 42.53583),-96.399473,42.535827,"(42.5358273, -96.3994734)",-96.399473,42.535827
3414,354389897,3200,jewish,United Orthodox Synagogue,POINT (-96.40308 42.50472),-96.403085,42.504716,"(42.5047164, -96.4030847)",-96.403085,42.504716
4165,965011815,3200,jewish,Temple Bethel EL,POINT (-90.67469 42.50874),-90.674686,42.508741,"(42.5087414, -90.6746856)",-90.674686,42.508741


In [ ]:
bus_route  = gpd.read_file('../GeoJsonFiles/bus_route.geojson', driver='ESRIJSON')

In [ ]:
landuse_polygon['landuse'].unique()

In [ ]:
landuse_polygon[landuse_polygon['landuse']=='retail'].plot()

In [ ]:
ames_shape

In [ ]:
ames_school_polygon = gpd.read_file('../GeoJsonFiles/school_polygon.geojson')

In [ ]:
ames_school_polygon.plot()

In [ ]:
ames_school_polygon.crs

In [ ]:
iowa-latest.osm.pbf

In [ ]:
G = ox.graph_from_place('Los Angeles, California', network_type='drive')
ox.plot_graph(G)

In [ ]:
city = ox.geocode_to_gdf('Ames, Iowa')
ax = ox.project_gdf(city).plot()
_ = ax.axis('off')

In [ ]:
G = ox.graph_from_place('Ames, Iowa', network_type='drive')
ox.plot_graph(G)

In [ ]:
city.crs

In [ ]:
city_wm = city.to_crs(epsg=3857)
landuse_crs = landuse_polygon.to_crs(epsg=3857)
schools_crs = schools.to_crs(epsg=3857)

In [ ]:

ax = landuse_crs.plot(figsize=(10, 10), alpha=0.5, edgecolor='k')
cx.add_basemap(ax)

In [ ]:
ames_school = ames_school_polygon.to_crs(epsg=3857)

ax = ames_school.plot(figsize=(10, 10), alpha=0.5, edgecolor='k')
cx.add_basemap(ax)

https://geopandas.readthedocs.io/en/latest/gallery/plotting_with_folium.html

https://python-visualization.github.io/folium/modules.html

https://www.earthdatascience.org/tutorials/introduction-to-leaflet-animated-maps/

https://autogis-site.readthedocs.io/en/latest/notebooks/L5/02_interactive-map-folium.html

https://github.com/mapbox/geojson-vt

for including in dash
https://medium.com/@shachiakyaagba_41915/integrating-folium-with-dash-5338604e7c56

https://community.plotly.com/t/how-to-render-map-created-by-folium-dynamically-and-get-the-clicked-data-information/40885/9

https://dash-leaflet.herokuapp.com/#map_click

https://medium.com/analytics-vidhya/brick-by-brick-build-a-multi-page-dashboard-dash-filters-dbec58d429d2

https://medium.com/analytics-vidhya/brick-by-brick-build-a-multi-page-dashboard-37912dd7d9e6

In [223]:
map = folium.Map(location = [42.0219, -93.6227], tiles='OpenStreetMap' , zoom_start = 12)
tooltip = "Click me!"
# folium.Marker(
#     [42.03047530489075, -93.60682804611989], popup="<i>That outlier</i>", tooltip=tooltip, 
#     icon=folium.Icon(icon="home"),
# ).add_to(map)

In [ ]:

# style_function = lambda x: {'fillColor': '#0000ff' if
#                            x['properties']['landuse']=='retail' else
#                             '#00ff00'}
# folium.GeoJson(landuse_polygon, name="Land Use", style_function= style_function, show=False).add_to(map)
# folium.GeoJson(schools_crs, name="Schools", show=False).add_to(map)
folium.GeoJson(pofw_crs, name="Churches", show=False).add_to(map)
# folium.GeoJson(poi_crs, name="Point of Interest", show=False).add_to(map)

# folium.plugins.FastMarkerCluster(gis_osm_pofw['geometry'], name="Churches").add_to(map)

#folium.Marker(pofw_crs, name="Churches", show=False).add_to(map)


# test = folium.GeoJson(pofw_crs, name="Churches", show=False,
#                     marker = Marker(radius = 3, # Radius in metres
#                                            weight = 0, #outline weight
#                                            fill_color = '#000000', 
#                                            fill_opacity = 1)
# #                ,
# #                     tooltip = folium.GeoJsonTooltip(fields = ['feeder'],
# #                                                     aliases=['Feeder: '],
# #                                                     style = ("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
# #                                                     sticky = True)
#                     ).add_to(map)
# # fillcolor: lambda x: {fillColor':'red' if x['properties']'Post_Cod_1'] == 'High' else 'green','fillOpacity':0.75}



# marker_cluster = folium.plugins.MarkerCluster(test)
# marker_cluster.add_to(map)

# folium.FeatureGroup(name=churches, show=False)

folium.LayerControl().add_to(map)


map

In [206]:
import os
import folium
import pandas as pd
 
marker_cluster = folium.plugins.MarkerCluster(location = pofw_crs.geometry[[1]], name = "Churches").add_to(map)
 

# for each in pofw_crs.iterrows():
# # #     popup = 'Add <b>test</b>'
# # #     print(list([each[1].GeoY,each[1].GeoX]))
# # #     print(list(OSGB36toWGS84(each[1]['GeoX'],each[1]['GeoY'])))
#      folium.Marker(list(pofw_crs.geometry[[each]])).add_to(marker_cluster)
map

TypeError: Object of type GeoDataFrame is not JSON serializable

In [205]:
gis_osm_pofw.geometry[[2]]


2    POINT (-92.00558 41.75245)
Name: geometry, dtype: geometry

In [ ]:


address='931 Maxwell ave, Ames Iowa'
geolocator = Nominatim(user_agent="ORoss_goldfish")
location = geolocator.geocode(address)
print(location.address)
print((location.latitude, location.longitude))

In [214]:

gis_osm_pofw["x"] = gis_osm_pofw["geometry"].apply(lambda geom: geom.x)
gis_osm_pofw["y"] = gis_osm_pofw["geometry"].apply(lambda geom: geom.y)

# Create a list of coordinate pairs
gis_osm_pofw['locations'] = list(zip(gis_osm_pofw["y"], gis_osm_pofw["x"]))

In [221]:
list(gis_osm_pofw['name'])

['Saint Anns Catholic Church',
 'Carmel Reformed Church',
 'First Lutheran',
 'Adventist Church',
 'Adventist Church',
 'Alice Church',
 'Allen Creek Chapel',
 'Alpine Log Church',
 'Altamont Church',
 'Amity Church',
 'Antioch Church',
 'Antioch Church',
 'Apostolic Church',
 'Armah Church',
 'Assumption Church',
 'Avondale Church',
 'Barclay Church',
 'Batavia Peoples Church',
 'Bay Church',
 'Beaver Creek Church',
 'Belmond Bible Church',
 'Bennezette Church',
 'Berea Church',
 'Bethany Church',
 'Bethany Church',
 'Bethel Chapel',
 'Bethel Church',
 'Bethel Church',
 'Bethel Church',
 'Bethel Church',
 'Bethel Church',
 'Bethel Church',
 'Bethel Church',
 'Bethel Church',
 'Bethel Church',
 'Bethel Church',
 'Bethel Church',
 'Bethel Church',
 'Bethel Grove Church',
 'Bethel Church',
 'Bethel Reform Church',
 'Bethelhem Church',
 'Bethlehem Church',
 'Bethlehem Church',
 'Bethlehem Church',
 'Beulah Church',
 'Bible Church',
 'Big Canoe Church',
 'Blair Chapel',
 'Immaculate Concep

In [ ]:
map = folium.Map(location = [42.0219, -93.6227], tiles='OpenStreetMap' , zoom_start = 12)
# Create a folium marker cluster
marker_cluster = MarkerCluster(locations = gis_osm_pofw['locations'],
                               popups=list(gis_osm_pofw['name']), 
                               name="Churches", show=False)

# Add marker cluster to map
marker_cluster.add_to(map)

folium.LayerControl().add_to(map)


# Show map
map

In [247]:
coords_1 = (gis_osm_pofw.y[1], gis_osm_pofw.x[1])
coords_2 = (gis_osm_pofw.y[2], gis_osm_pofw.x[1])

print(geodesic(coords_1, coords_2).miles)

94.86142256612712


AttributeError: 'tuple' object has no attribute 'raw'

In [243]:
gis_osm_pofw['lon'] = gis_osm_pofw['geometry'].x
gis_osm_pofw['lat'] = gis_osm_pofw['geometry'].y

In [244]:
gis_osm_pofw

,osm_id,code,fclass,name,geometry,x,y,locations,lon,lat
0,159451514,3102,christian_catholic,Saint Anns Catholic Church,POINT (-95.19847 42.06358),-95.198469,42.063582,"(42.063582, -95.198469)",-95.198469,42.063582
1,245093558,3100,christian,Carmel Reformed Church,POINT (-96.23419 43.12680),-96.234185,43.126796,"(43.1267965, -96.2341852)",-96.234185,43.126796
2,278005220,3104,christian_lutheran,First Lutheran,POINT (-92.00558 41.75245),-92.005576,41.752452,"(41.7524524, -92.0055758)",-92.005576,41.752452
3,354294506,3100,christian,Adventist Church,POINT (-94.97665 41.61277),-94.976654,41.612766,"(41.6127655, -94.9766544)",-94.976654,41.612766
4,354294510,3100,christian,Adventist Church,POINT (-95.02333 43.32830),-95.023325,43.328295,"(43.3282951, -95.0233252)",-95.023325,43.328295
...,...,...,...,...,...,...,...,...,...,...
4266,8381432075,3105,christian_methodist,Maple Grove Church,POINT (-93.83763 41.58060),-93.837625,41.580597,"(41.5805966, -93.8376251)",-93.837625,41.580597
4267,8406214777,3102,christian_catholic,Rodion Kabak,POINT (-93.56995 41.23816),-93.569953,41.238163,"(41.2381631, -93.5699526)",-93.569953,41.238163
4268,8436986197,3100,christian,The Church of Jesus Christ of Latter-day Saints,POINT (-95.76766 41.09255),-95.767661,41.092551,"(41.0925505, -95.7676613)",-95.767661,41.092551
4269,8830485935,3100,christian,Capitol Hill Christian Church,POINT (-93.57153 41.62560),-93.571527,41.625604,"(41.6256037, -93.5715266)",-93.571527,41.625604


42.1089
41.9503

-93.7638
-93.4449